<a href="https://colab.research.google.com/github/saghar-trade/Time-series/blob/main/Walk_forward_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.holtwinters import ExponentialSmoothing


In [37]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/air_passengers.csv' , index_col='Month' , parse_dates=True)
df.tail()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Passengers
Month,
1960-08-01,606
1960-09-01,508
1960-10-01,461
1960-11-01,390
1960-12-01,432


In [38]:
df.isna().sum()

,0
Passengers,0


In [39]:
df.index.freq='MS'

In [40]:
df.shape

(144, 1)

In [41]:
#Forecast horizone is 12
h=12
steps=10
Ntest=len(df)-h-steps+1

In [65]:
#configuration hyperparameter
Trend_type_list=['add','mul']
Seasonal_type_list=['add','mul']
damped_trend_list=[True, False]
init_method_list=['estimated', 'heuristic','legacy-heuristic']
use_boxcox_list=[True, False,0]



In [61]:
def walkforward(
 Trend_type,Seasonal_type , damped_trend , init_method ,use_boxcox,debug=False
):
  errors=[]
  seen_last=False
  step_completed=0
  for end_of_train in range(Ntest,len(df)-h+1):
    train=df.iloc[:end_of_train]
    test=df.iloc[end_of_train:end_of_train+h]
    if test.index[-1]==df.index[-1]:
      seen_last=True
    step_completed +=1
    HW = ExponentialSmoothing(train['Passengers'],
        initialization_method=init_method,
        trend=Trend_type,
        damped_trend=damped_trend ,
        seasonal=Seasonal_type,
        seasonal_periods=12,
        use_boxcox=use_boxcox
        )
    res_hw=HW.fit()
    #compute error
    fcast=res_hw.forecast(h)
    error=mean_squared_error(test['Passengers'], fcast)
    errors.append(error)
  if debug:
     print('seen last:',seen_last)
     print('step completed:',step_completed)
  return np.mean(errors)





In [62]:
#test our function
walkforward('add','add' , False , 'legacy-heuristic',0,debug=True)


seen last: True
step completed: 10


np.float64(1448.227487951414)

In [66]:
#iterate through all options
touple_of_option_list=(Trend_type_list,
Seasonal_type_list,
damped_trend_list,
init_method_list,
use_boxcox_list)
for x in itertools.product(*touple_of_option_list):
  print(x)

('add', 'add', True, 'estimated', True)
('add', 'add', True, 'estimated', False)
('add', 'add', True, 'estimated', 0)
('add', 'add', True, 'heuristic', True)
('add', 'add', True, 'heuristic', False)
('add', 'add', True, 'heuristic', 0)
('add', 'add', True, 'legacy-heuristic', True)
('add', 'add', True, 'legacy-heuristic', False)
('add', 'add', True, 'legacy-heuristic', 0)
('add', 'add', False, 'estimated', True)
('add', 'add', False, 'estimated', False)
('add', 'add', False, 'estimated', 0)
('add', 'add', False, 'heuristic', True)
('add', 'add', False, 'heuristic', False)
('add', 'add', False, 'heuristic', 0)
('add', 'add', False, 'legacy-heuristic', True)
('add', 'add', False, 'legacy-heuristic', False)
('add', 'add', False, 'legacy-heuristic', 0)
('add', 'mul', True, 'estimated', True)
('add', 'mul', True, 'estimated', False)
('add', 'mul', True, 'estimated', 0)
('add', 'mul', True, 'heuristic', True)
('add', 'mul', True, 'heuristic', False)
('add', 'mul', True, 'heuristic', 0)
('add

In [68]:
best_score=float('inf')
best_options=None
for x in itertools.product(*touple_of_option_list):
  score=walkforward(*x)
  if score < best_score:
    print('best score so far:', score)
    best_score=score
    best_options=x





best score so far: 412.70439433481533
best score so far: 397.51642337228634
best score so far: 368.8467123043263
best score so far: 320.68497356823144
best score so far: 307.9434782402395
best score so far: 300.95995233097807


KeyboardInterrupt: 

In [69]:
print('best score:', best_score)

best score: 300.95995233097807


In [72]:
Trend_type,Saesonal_type , damped_trend , init_method ,use_boxco=best_options
print("Trend_type :",  Trend_type)
print('Saesonal_type:',  Saesonal_type)
print('damped_trend:', damped_trend)
print('init_method :', init_method)
print('use_boxco :', use_boxco)




Trend_type : mul
Saesonal_type: add
damped_trend: True
init_method : legacy-heuristic
use_boxco : False
